# Tabula Muris Senis Analysis

## Install & Load Packages ------

In [ ]:
import scanpy as sc
import seaborn as sns
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as stats
import os
import sys
import warnings
from matplotlib.patches import Patch

# suppress warnings
warnings.filterwarnings('ignore')

## Load Dataset ------

In [ ]:
# Dataset link: https://figshare.com/articles/dataset/Processed_files_to_use_with_scanpy_/8273102?file=23936555
h5ad = "/oak/stanford/groups/ckuo/Shawn_shared/Tabula_Muris_Analysis/tabula_muris_senis_combined.h5ad"
adata = sc.read_h5ad(h5ad)
sc.set_figure_params(dpi=300)
adata.obs

## Data Preparation & Preprocessing

In [ ]:
# Identify highly-variable genes: https://www.nature.com/articles/nbt.3192
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
sc.pl.highly_variable_genes(adata)

# View highly variable genes 
highly_variable_genes = adata[:, adata.var.highly_variable].var # 3135 highly-variable genes
highly_variable_genes.to_csv('highly_variable_genes.csv')

## Principal Component Analysis -----

In [ ]:
# Reduce the dimensionality of the data
sc.tl.pca(adata, svd_solver='arpack')

In [ ]:
# Inspect the contribution of single PCs to the total variance in the data
sc.pl.pca_variance_ratio(adata, log=True)

In [ ]:
# Compute the neighborhood graph of cells using the PCA representation of the data matrix
sc.pp.neighbors(adata, n_neighbors=15, n_pcs=30, method='umap')

## Exploratory Data Analysis -----

In [ ]:
sc.pl.umap(adata, color = "tissue", save = '_ms_all_cells')

In [ ]:
sc.pl.umap(adata, color='leiden', cmap='RdYlBu_r', use_raw=False, save='_ms_all_leiden')

In [ ]:
# Dotplot to visualize markers
markers = ['Sox10', 'Ncam1','Plp1', 'S100b','Gfra3', 'Gfap']
sc.pl.dotplot(adata, markers, groupby='leiden', swap_axes=True, save = "ms_markers_dotplot.pdf", dot_max=1, use_raw=False)

In [ ]:
# Subset dataset to cells assigned to cluster 32
cluster32 = adata[adata.obs['leiden'] == '32']
cluster32.obs.groupby(['tissue']).size().to_frame(name='counts').to_csv('cluster32_tissue_df.csv')
cluster32.obs.groupby(['cell_ontology_class']).size().to_frame(name='counts').to_csv('cluster32_cell_class_df.csv')

In [ ]:
def highlightCluster(adata, cluster):
    adata.obs['glial_highlight'] = 'non_glia'
    adata.obs['glial_highlight'] = adata.obs['glial_highlight'].astype('category')
    adata.obs['glial_highlight'] = adata.obs['glial_highlight'].cat.set_categories(['non_glia', 'glia_candidates'])
    adata.obs['glial_highlight'].loc[adata.obs['leiden'] == str(cluster)] = 'glia_candidates'
    return adata

In [ ]:
# Highlight cluster 32 on UMAP
adata_highlight_32 = highlightCluster(adata, 32)
sc.pl.umap(adata_highlight_32, color='glial_highlight', cmap='RdYlBu_r', palette={'non_glia':"grey", 'glia_candidates':'orange'}, use_raw=False, save='_ms_all_cells_cluster_32')

In [ ]:
sc.pl.umap(adata, color='Sox10', cmap='RdYlBu_r', use_raw=False, save='_ms_all_cells_Sox10_colored') # not highly variable gene?

In [ ]:
sc.pl.umap(adata, color='Cryab', cmap='RdYlBu_r', use_raw=False, save='_ms_all_cells_Cryab_colored')

In [ ]:
sc.pl.umap(adata, color='Cdh19', cmap='RdYlBu_r', use_raw=False, save='_ms_all_cells_Cdh19_colored')

In [ ]:
sc.pl.umap(adata, color='Plp1', cmap='RdYlBu_r', use_raw=False, save='_ms_all_cells_Plp1_colored') # not highly variable gene?

In [ ]:
sc.pl.umap(adata, color='S100b', cmap='RdYlBu_r', use_raw=False, save='_ms_all_cells_S100b_colored')

In [ ]:
sc.pl.umap(adata, color='Ncam1', cmap='RdYlBu_r', use_raw=False, save='_ms_all_cells_Ncam1_colored') # not highly variable gene?

In [ ]:
sc.pl.umap(adata, color='Gfra3', cmap='RdYlBu_r', use_raw=False, save='_ms_all_cells_Gfra3_colored') # not highly variable gene?

In [ ]:
sc.pl.umap(adata, color='Gfap', cmap='RdYlBu_r', use_raw=False, save='_ms_all_cells_Gfap_colored')

In [ ]:
sc.pl.umap(adata, color='Mbp', cmap='RdYlBu_r', use_raw=False, save='_ms_all_cells_Mbp_colored')

In [ ]:
# Define a function for making a violin plot, given the tissue and gene
def violin_plot(adata, tissue, gene):
    tissue_subset = adata[adata.obs['tissue'] == tissue]
    with mpl.rc_context({"figure.figsize": (10, 5), "figure.dpi": (300)}):
        plot = sc.pl.violin(tissue_subset, [gene], groupby='leiden', use_raw = False, rotation=90, 
                            ylabel=tissue+'_'+gene, show=False, save='_ms_'+tissue+'_'+gene)
    return plot 

# Violin plot for marker genes of Limb_Muscle
violin_plot(adata, "Limb_Muscle", "Sox10")
violin_plot(adata, "Limb_Muscle", "S100b")
violin_plot(adata, "Limb_Muscle", "Plp1")
violin_plot(adata, "Limb_Muscle", "Ncam1")
violin_plot(adata, "Limb_Muscle", "Gfap")
violin_plot(adata, "Limb_Muscle", "Nrxn1")
violin_plot(adata, "Limb_Muscle", "Cryab")
violin_plot(adata, "Limb_Muscle", "Scn7a")
violin_plot(adata, "Limb_Muscle", "Mbp")
violin_plot(adata, "Limb_Muscle", "Mpz")

## Differential expression: finding marker genes ------ 

In [ ]:
# Get characterizing genes for cluster 32 vs the rest
sc.tl.rank_genes_groups(adata, 'leiden', groups=['32'], reference='rest', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, save='_ms_32_vs_rest', sharey=False)

In [ ]:
# Dotplot for cluster 32 ranked genes
sc.pl.rank_genes_groups_dotplot(adata, n_genes=100, groupby="leiden", save='_ms_rgg_cluster32_dotplot', use_raw=False)

In [ ]:
# Get rgg result into dataframe
sc.get.rank_genes_groups_df(adata, group='32').to_csv('rgg_32.csv')
sc.get.rank_genes_groups_df(adata, group='32')

<hr>

In [ ]:
# Analysis of Gpm6b
gpm6b_index = np.where(adata.raw.var_names == 'Gpm6b')[0][0]
gpm6b_arr = adata.raw.X.toarray()[:,gpm6b_index]
adata.obs['Gpm6b_col'] = gpm6b_arr

In [ ]:
# Visualize Gpm6b on ViolinPlot
sc.pl.violin(adata, ['Gpm6b_col'], groupby='leiden', use_raw = False, save='_ms_violin_across_groups_Gpm6b')
sc.set_figure_params(figsize=(20, 20))

In [ ]:
# Visualize Gpm6b on UMAP
sc.pl.umap(adata, color=['Gpm6b'], cmap='RdYlBu_r', use_raw=False, save='_ms_32_gpm6b')

<hr>

In [ ]:
# Analysis of Cryab
cryab_index = np.where(adata.raw.var_names == 'Cryab')[0][0]
cryab_arr = adata.raw.X.toarray()[:,cryab_index]
adata.obs['Cryab_col'] = cryab_arr

In [ ]:
# Visualize Cryab on ViolinPlot
sc.pl.violin(adata, ['Cryab_col'], groupby='leiden', use_raw = False, save='_ms_violin_across_groups_Cryab')
sc.set_figure_params(figsize=(20, 20))

In [ ]:
# Visualize Cryab on UMAP
sc.pl.umap(adata, color=['Cryab'], cmap='RdYlBu_r', use_raw=False, save='_ms_32_cryab')

<hr>

In [ ]:
# Analysis of Scd2
scd2_index = np.where(adata.raw.var_names == 'Scd2')[0][0]
scd2_arr = adata.raw.X.toarray()[:,scd2_index]
adata.obs['Scd2_col'] = scd2_arr

In [ ]:
# Visualize Scd2 on ViolinPlot
sc.pl.violin(adata, ['Scd2_col'], groupby='leiden', use_raw = False, save='_ms_violin_across_groups_Scd2')
sc.set_figure_params(figsize=(20, 20))

In [ ]:
# Visualize Scd2 on UMAP
sc.pl.umap(adata, color=['Scd2'], cmap='RdYlBu_r', use_raw=False, save='_ms_32_scd2')

## Sub-set Cluster 32 (No CNS) -----

In [ ]:
# Filter out brain tissue from cluster 32
cns_mask = (cluster32.obs['tissue'] != "Brain_Non-Myeloid") & (cluster32.obs['tissue'] != "Brain_Myeloid")
cns_mask[cns_mask == False] # 3176 CNS cells in cluster 32

cluster32_no_cns = cluster32[cns_mask]
cluster32_no_cns.obs # 1124 cells in cluster 32, without CNS (

In [ ]:
# Run leiden on sub-cluster 32
sc.tl.leiden(cluster32_no_cns, resolution = 0.01, key_added = "leiden_0.01")
sc.tl.leiden(cluster32_no_cns, resolution = 0.05, key_added = "leiden_0.05")
sc.tl.leiden(cluster32_no_cns, resolution = 0.1, key_added = "leiden_0.1")
sc.tl.leiden(cluster32_no_cns, resolution = 0.2, key_added = "leiden_0.2")

sc.pl.umap(cluster32_no_cns, color=['leiden_0.01', 'leiden_0.05', 'leiden_0.1', 'leiden_0.2'], use_raw=False, save='_ms_cluster32_no_cns_leiden')
#sc.set_figure_params(figsize=(10, 15))

In [ ]:
# Choose resolution leiden_0.05. Inspect number of cells in each cluster
cluster32_no_cns.obs['leiden_0.05'].value_counts().sort_index().plot(kind="bar")

# Only 1 cell from clusters 7~20, we will filter them out
cluster32_no_cns_filtered = cluster32_no_cns[pd.to_numeric(cluster32_no_cns.obs['leiden_0.05']) < 7]
cluster32_no_cns_filtered # 1105 cells
sc.pl.umap(cluster32_no_cns_filtered, color='leiden_0.05', palette="Set1", use_raw=False, save='_ms_cluster32_no_cns_filtered_0.05')
sc.set_figure_params(dpi=300)

In [ ]:
# Get characterizing genes within re-clustered 32
sc.tl.rank_genes_groups(cluster32_no_cns_filtered, 'leiden_0.05', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(cluster32_no_cns_filtered, n_genes=10, save='_ms_32_recluster_rank', sharey=False)

In [ ]:
# Get rank_genes_groups into dataframe
sc.get.rank_genes_groups_df(cluster32_no_cns_filtered, group='0').to_csv('recluster32_rgg_0.csv')
sc.get.rank_genes_groups_df(cluster32_no_cns_filtered, group='1').to_csv('recluster32_rgg_1.csv')
sc.get.rank_genes_groups_df(cluster32_no_cns_filtered, group='2').to_csv('recluster32_rgg_2.csv')
sc.get.rank_genes_groups_df(cluster32_no_cns_filtered, group='3').to_csv('recluster32_rgg_3.csv')
sc.get.rank_genes_groups_df(cluster32_no_cns_filtered, group='4').to_csv('recluster32_rgg_4.csv')
sc.get.rank_genes_groups_df(cluster32_no_cns_filtered, group='5').to_csv('recluster32_rgg_5.csv')
sc.get.rank_genes_groups_df(cluster32_no_cns_filtered, group='6').to_csv('recluster32_rgg_6.csv')

In [ ]:
# UMAP of filtered cluster32, colored by tissue 
sc.pl.umap(cluster32_no_cns_filtered, color='tissue', save='_ms_cluster32_filtered_no_cns_tissue')

# Count of filtered cluster32 for each tissue type 
cluster32_no_cns_filtered.obs.groupby(['tissue']).size().to_frame(name='counts').to_csv('cluster32_filtered_no_cns_tissue_df.csv')
cluster32_no_cns_filtered.obs.groupby(['tissue']).size().to_frame(name='counts')

In [ ]:
# Highlight genes on filtered cluster32 (no CNS) UMAP
sc.pl.umap(cluster32_no_cns_filtered, color='Sox10', cmap='RdYlBu_r', use_raw=False, save='_ms_cluster32_no_cns_sox10_HL')
sc.pl.umap(cluster32_no_cns_filtered, color='S100b', cmap='RdYlBu_r', use_raw=False, save='_ms_cluster32_no_cns_s100b_HL')
sc.pl.umap(cluster32_no_cns_filtered, color='Plp1', cmap='RdYlBu_r', use_raw=False, save='_ms_cluster32_no_cns_plp1_HL')
sc.pl.umap(cluster32_no_cns_filtered, color='Ncam1', cmap='RdYlBu_r', use_raw=False, save='_ms_cluster32_no_cns_ncam1_HL')
sc.pl.umap(cluster32_no_cns_filtered, color='Gfap', cmap='RdYlBu_r', use_raw=False, save='_ms_cluster32_no_cns_gfap_HL')
sc.pl.umap(cluster32_no_cns_filtered, color='Nrxn1', cmap='RdYlBu_r', use_raw=False, save='_ms_cluster32_no_cns_nrxn1_HL')
sc.pl.umap(cluster32_no_cns_filtered, color='Erbb3', cmap='RdYlBu_r', use_raw=False, save='_ms_cluster32_no_cns_erbb3_HL')
sc.pl.umap(cluster32_no_cns_filtered, color='Cryab', cmap='RdYlBu_r', use_raw=False, save='_ms_cluster32_no_cns_cryab_HL')
sc.pl.umap(cluster32_no_cns_filtered, color='Scn7a', cmap='RdYlBu_r', use_raw=False, save='_ms_cluster32_no_cns_scn7a_HL')

In [ ]:
# Function for generating a heatmap for specified tissue
def snsHeatmap(adata, tissue):
    adata.obs['barcodes'] = adata.obs.index
    def sub_genes(adata, tissue):
        sub = adata[adata.obs['tissue']==tissue]
        sub = sub[:, ['Sox10','Plp1','S100b','Ncam1','Gfap','Pllp','Mpz','Mbp','Pmp22','Cdkn1c','Scn7a',
                      'Gfra3','Apoe','Nrxn1','Col1a1','Acta2','Epcam','Cdh1','Krt5','Krt14','Ptprc','Itgam',
                      'Cd3e','Cd19','Mitf','Dct','Mlana','Pmel','Tyr','Tyrp1']]
        sub.obs['barcodes']=sub.obs.index
        sub.obs['barcodes']=sub.obs['barcodes'].astype('category')
        display(sub)
        return(sub)
    sub = sub_genes(adata,tissue)
    heatmap = sns.clustermap(sub.X.todense(), yticklabels = list(sub.obs['barcodes']), xticklabels= list(sub.var.index), col_cluster = False, cmap='RdYlBu_r')
    heatmap.tick_params(axis='both', which='major', labelsize=3)
    fig = heatmap.fig
    fig.savefig("_ms_" + tissue + "_cluster32_filtered_heatmap.pdf")

In [ ]:
# Apply heatmap function on specified tissues
snsHeatmap(cluster32_no_cns_filtered, "Limb_Muscle")
snsHeatmap(cluster32_no_cns_filtered, "Bladder")
snsHeatmap(cluster32_no_cns_filtered, "Trachea")
snsHeatmap(cluster32_no_cns_filtered, "Fat")
snsHeatmap(cluster32_no_cns_filtered, "Heart")
snsHeatmap(cluster32_no_cns_filtered, "Kidney")
snsHeatmap(cluster32_no_cns_filtered, "Lung")

## Filter Tissues & Cells of Re-Clustered 32 (No CNS) -----

In [ ]:
# Filter out tissues of interest (7 in total)
filtered_tissue = cluster32_no_cns_filtered[cluster32_no_cns_filtered.obs.loc[cluster32_no_cns_filtered.obs['tissue'].isin(['Limb_Muscle', 'Bladder', 'Trachea', 'Fat', 'Heart', 'Kidney', 'Lung'])].index]
filtered_tissue.obs['tissue']

In [ ]:
# Filter out barcodes of fat, heart, trachea that are not glial (selected from heatmap analysis)
remove_barcode = pd.read_csv('remove_barcode.csv', header=None)
remove_barcode[0].to_numpy()
filtered_barcodes = filtered_tissue[filtered_tissue.obs.loc[~filtered_tissue.obs['barcodes'].isin(remove_barcode[0].to_numpy())].index]

In [ ]:
# Filter out barcodes of kidney that are not glial (selected from heatmap analysis)
kidney_barcode = pd.read_csv('kidney_barcode.csv', header=None) # read in kidney barcodes that we want to keep
kidney_barcode[0].to_numpy()
kidney = filtered_barcodes[filtered_barcodes.obs['tissue'] == 'Kidney'] # subset out kidney tissue
kidney_filter = kidney[kidney.obs.loc[~kidney.obs['barcodes'].isin(kidney_barcode[0].to_numpy())].index].obs['barcodes'] # get the kidney barcodes that we want to filter out

cluster32_glial = filtered_barcodes[filtered_barcodes.obs.loc[~filtered_barcodes.obs['barcodes'].isin(kidney_filter.to_numpy())].index] # 435 cells
cluster32_glial.obs.to_csv('cluster32_glial.csv')

In [ ]:
cluster32_glial.write_h5ad("cluster32_glial.h5ad")

## Plot Outputs of Subsetted Glial Cells From Cluster 32 (No CNS)

In [ ]:
# UMAP of filtered cluster32, colored by tissue 
sc.pl.umap(cluster32_glial, color='tissue', palette="Set1", save='_ms_glial_tissue')

# Count of filtered cluster32 for each tissue type 
cluster32_glial.obs.groupby(['tissue']).size().to_frame(name='counts').to_csv('glial_tissue_df.csv')
cluster32_glial.obs.groupby(['tissue']).size().to_frame(name='counts')

In [ ]:
# Highlight leiden_0.05 cluster on UMAP
sc.pl.umap(cluster32_glial, color='leiden_0.05', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_leiden_0.05')

In [ ]:
# Highlight selected genes on subsetted glial cells UMAP
sc.pl.umap(cluster32_glial, color='Sox10', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_sox10_HL')
sc.pl.umap(cluster32_glial, color='S100b', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_s100b_HL')
sc.pl.umap(cluster32_glial, color='Plp1', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_plp1_HL')
sc.pl.umap(cluster32_glial, color='Ncam1', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_ncam1_HL')
sc.pl.umap(cluster32_glial, color='Gfap', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_gfap_HL')
sc.pl.umap(cluster32_glial, color='Nrxn1', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_nrxn1_HL')
sc.pl.umap(cluster32_glial, color='Erbb3', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_erbb3_HL')
sc.pl.umap(cluster32_glial, color='Cryab', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_cryab_HL')
sc.pl.umap(cluster32_glial, color='Scn7a', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_scn7a_HL')
sc.pl.umap(cluster32_glial, color='Mbp', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_mbp_HL')
sc.pl.umap(cluster32_glial, color='Mpz', cmap='RdYlBu_r', use_raw=False, save='_ms_glial_mpz_HL')

In [ ]:
# Highlight final glial on UMAP
def glial_umap(adata, ind):
    adata.obs['final_glial'] = 'non_glial'
    adata.obs['final_glial'] = adata.obs['final_glial'].astype('category')
    adata.obs['final_glial'] = adata.obs['final_glial'].cat.set_categories(['non_glial', 'glial'])
    adata.obs['final_glial'].loc[ind] = 'glial'
    return sc.pl.umap(adata, color='final_glial', palette={"non_glial":"grey", "glial":'orange'}, use_raw=False, save='_ms_glial_highlight')

glial_umap(adata, list(cluster32_glial.obs.index))

In [ ]:
# Get characterizing genes for glial cluster vs the rest
sc.tl.rank_genes_groups(adata, 'final_glial', method='wilcoxon', use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, save='_ms_glial_vs_rest_rank', sharey=False)

In [ ]:
# Get characterizing genes for glial cluster vs the rest into a dataframe 
sc.get.rank_genes_groups_df(adata, group='non_glial').to_csv('non_glial_rgg.csv')
sc.get.rank_genes_groups_df(adata, group='glial').to_csv('glial_rgg.csv')

In [ ]:
# Plot top differential genes on UMAP
sc.pl.umap(adata, color=['Gpm6b', 'Art3', 'Prnp', 'Cryab', 'Fxyd1', 'Kcna1'], cmap='RdYlBu_r', use_raw=False, save='_ms_top_glial')

## Annotation Dataframe -----

In [ ]:
# Glial cells (435 rows) dataframe from R clustering
annotation_df = pd.read_excel('metadata_clusters_seurat.xlsx')
annotation_df.drop(columns=annotation_df.columns[-4:], axis=1, inplace=True)
#annotation_df = annotation_df.rename(columns={'Unnamed: 0': 'index'})
annotation_df

In [ ]:
# Label glial_class based on seurat_clusters
class_dictionary = {0 : 'non-myelinating', 1 : 'terminal', 2:'myelinating' }
annotation_df["glial_class"] = annotation_df["seurat_clusters"].map(class_dictionary)
annotation_df

In [ ]:
# Set order of tissue and glial_class columns
annotation_df['tissue'] = pd.Categorical(annotation_df['tissue'], ['Limb_Muscle', 'Heart', 'Trachea', 'Fat', 'Bladder', 'Lung', 'Kidney'])
annotation_df['glial_class'] = pd.Categorical(annotation_df['glial_class'], ['myelinating', 'non-myelinating', 'terminal'])

# Sort dataframe by tissue (descending cell #) then glial_class
annotation_df = annotation_df.sort_values(by=['tissue','glial_class'], ignore_index=True)
annotation_df

In [ ]:
annotation_df.to_csv('glial_metadata.csv')

In [ ]:
# Label glial vs nonglial in the adata; Merge with annotation_df to get glial_df column
adata.obs['glial'] = np.where(adata.obs.index.isin(annotation_df['index']), 'glial', 'non_glial')
glial_adata = adata[adata.obs['glial'] == 'glial']
glial_adata.obs = glial_adata.obs.merge(annotation_df, on='index', how='outer')
glial_adata.obs.index = glial_adata.obs['index']
glial_adata.obs

In [ ]:
adata.write_h5ad("adata_labeled.h5ad")

## Visualization Outputs -----

### Figure for number/percentage of each glial_class across all tissue types

In [ ]:
# Tabulate count of each tissue type organized by glial class
annotation_df.groupby(["tissue", "glial_class"]).size()

In [ ]:
# Create table of percentage of glial_class across all 7 tissue types 
glial_class_prop = pd.crosstab(index=annotation_df['tissue'],
                             columns=annotation_df['glial_class'],
                             normalize="index") * 100
glial_class_prop.to_csv('glial_class_prop.csv')
glial_class_prop

In [ ]:
# Create stacked barplot of percentage of glial_class across all 7 tissue types 
glial_class_prop.plot(kind='bar', 
                      cmap='gray',
                      edgecolor = "black",
                      stacked=True)
plt.legend(bbox_to_anchor=(1.1, 1.25))
plt.xticks(rotation=25, fontsize=7)
plt.ylabel("% Glial Cells")
plt.autoscale()
plt.grid(False)
plt.savefig("glial_class_prop.pdf", bbox_inches = "tight")

### Heatmap for the finalized 435 glial cells across all tissue types

In [ ]:
# Function for generating a heatmap for specified tissue of finalized glial cells
def snsHeatmap(adata, tissue):
    adata.obs['barcodes'] = adata.obs.index
    adata.obs.index = adata.obs.index.astype(str)
    def sub_genes(adata, tissue):
        sub = adata[adata.obs['tissue_y']==tissue]
        sub = sub[:, ['Sox10','Plp1','S100b','Gpm6b','Prnp','Cryab','Kcna1','Igfbp7',
                      'Scn7a','Gfra3','Apoe','Cldn19','Prx','Pllp','Bcas1','Mbp','Mpz',
                      'Tinagl1','Gpc3','Bche','Cpm','Pla2g7','Kcnn4','Igfbp4','Sod3']]
        sub.obs['barcodes_y']=sub.obs.index
        sub.obs['barcodes_y']=sub.obs['barcodes_y'].astype('category')
        display(sub)
        return(sub)
    sub = sub_genes(adata,tissue)
    
    # Sort the order of glial_class
    sub.obs['glial_class'] = pd.Categorical(sub.obs['glial_class'], 
                                            categories=['terminal', 'myelinating', 'non-myelinating'], ordered=True)
    sub = sub[sub.obs.sort_values(['glial_class']).index]
    glial_class_col = sub.obs["glial_class"]
    
    # Map custom colors of glial_class
    lut = {'non-myelinating': '#F8766D', 'myelinating': '#619CFF', 'terminal': '#00BA38'}
    row_colors = glial_class_col.map(lut).to_numpy()
    
    # Generate the heatmap and adjust tick labels
    heatmap = sns.clustermap(sub.X.todense(), yticklabels = list(sub.obs['barcodes_y']), xticklabels= list(sub.var.index), 
                              col_cluster = False, row_cluster = False, cmap='RdYlBu_r', row_colors=row_colors)
    heatmap.tick_params(axis='both', which='major', labelsize=3)
    
    # Legend for glial_class
    handles = [Patch(facecolor=lut[name]) for name in lut]
    plt.legend(handles, lut, title='glial_class', bbox_to_anchor=(0.7, 1), 
               bbox_transform=plt.gcf().transFigure, loc='upper right')
    
    fig = heatmap.fig
    fig.savefig("_ms_" + tissue + "_glial_adata_heatmap.pdf")
    
    return glial_class_col

In [ ]:
# Apply heatmap function on specified tissues
Limb_Muscle_heatmap = snsHeatmap(glial_adata, "Limb_Muscle")
Bladder_heatmap = snsHeatmap(glial_adata, "Bladder")
Trachea_heatmap = snsHeatmap(glial_adata, "Trachea")
Fat_heatmap = snsHeatmap(glial_adata, "Fat")
Heart_heatmap = snsHeatmap(glial_adata, "Heart")
Kidney_heatmap = snsHeatmap(glial_adata, "Kidney")
Lung_heatmap = snsHeatmap(glial_adata, "Lung")

## Get glial df in the same order as the clustermaps

In [ ]:
reord_barcodes = list(Limb_Muscle_heatmap[::-1].index) + list(Trachea_heatmap[::-1].index) + list(Heart_heatmap[::-1].index) + list(Bladder_heatmap[::-1].index) + list(Fat_heatmap[::-1].index) + list(Lung_heatmap[::-1].index) + list(Kidney_heatmap[::-1].index)

In [ ]:
df1 = glial_adata.obs.set_index('index')
df1 = df1.reindex(reord_barcodes)

In [ ]:
final_glial_df = pd.DataFrame().assign(Cell_ID=df1['barcodes'], Age=df1['age_x'], Source_Description=df1['cell_x'], 
                                       Cell_Ontology_Class=df1['cell_ontology_class_x'],Free_Annotation=df1['free_annotation_x'], 
                                       Method=df1['method_x'],Mouse_ID=df1['mouse.id_x'], n_counts=df1['n_counts_x'], 
                                       Tissue=df1['tissue_y'],Tissue_Free_Annotation=df1['tissue_free_annotation_x'], 
                                       Glial_Class=df1['glial_class']) 
final_glial_df.to_csv('final_glial.csv')

<br>
<br>
<br>
<hr>
<br>
<br>
<br>